In [1]:
!pip install mysql-connector-python sqlalchemy

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import pandas as pd
import mysql.connector

# 1. Define credentials
config = {
  'user': 'root',
  'password': 'Shreya@4546', 
  'host': '127.0.0.1',
  'database': 'retail_project',
  'raise_on_warnings': True
}

try:
    # 2. Establish the connection
    cnx = mysql.connector.connect(**config)
    
    # 3. Pull the data into a DataFrame
    query = "SELECT * FROM rfm_metrics"
    df = pd.read_sql(query, cnx)
    
    print("Success! Connection established.")
    print(f"Total Rows Imported: {len(df)}")
    
    # 4. Close the connection
    cnx.close()

except mysql.connector.Error as err:
    print(f"Error: {err}")


df.head()

Success! Connection established.
Total Rows Imported: 974


C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,CustomerID,recency,frequency,monetary
0,,2,177,269825.17
1,12347,33,1,711.79
2,12348,24,1,892.80
3,12370,23,2,1868.02
4,12377,20,1,1001.52


In [7]:
# 1. Create the 1-5 Scores
# Recency: Lower is better (more recent), so 5 is assigned to the smallest values
df['R_Score'] = pd.qcut(df['recency'], 5, labels=[5, 4, 3, 2, 1])

# Frequency: Higher is better, so 5 is assigned to the largest values
# I use rank(method='first') because many customers might have the same low frequency
df['F_Score'] = pd.qcut(df['frequency'].rank(method='first'), 5, labels=[1, 2, 3, 4, 5])

# Monetary: Higher is better
df['M_Score'] = pd.qcut(df['monetary'], 5, labels=[1, 2, 3, 4, 5])

# 2. Defining the Customer Segments based on R and F scores
def segment_me(row):
    r, f = int(row['R_Score']), int(row['F_Score'])
    if r >= 4 and f >= 4:
        return 'Champions'
    elif r >= 3 and f >= 3:
        return 'Loyal Customers'
    elif r >= 4 and f <= 2:
        return 'New/Recent Customers'
    elif r <= 2 and f >= 4:
        return 'At Risk'
    elif r <= 2 and f <= 2:
        return 'Lost'
    else:
        return 'Average/Potential'

df['Segment'] = df.apply(segment_me, axis=1)


print(df['Segment'].value_counts())
df.head()

Champions               215
Lost                    183
Loyal Customers         180
Average/Potential       168
New/Recent Customers    130
At Risk                  98
Name: Segment, dtype: int64


,CustomerID,recency,frequency,monetary,R_Score,F_Score,M_Score,Segment
0,,2,177,269825.17,5,5,5,Champions
1,12347,33,1,711.79,2,1,4,Lost
2,12348,24,1,892.80,4,1,5,New/Recent Customers
3,12370,23,2,1868.02,4,4,5,Champions
4,12377,20,1,1001.52,4,1,5,New/Recent Customers


In [10]:
# Removing the blank/null CustomerIDs
df_clean = df.dropna(subset=['CustomerID'])

In [11]:
#Average Recency, Frequency, and Monetary for each segment
segment_summary = df_clean.groupby('Segment').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean'
}).round(2)

print(segment_summary)

                      recency  frequency  monetary
Segment                                           
At Risk                 34.27       2.03    904.38
Average/Potential       31.25       1.00    355.24
Champions               11.06       3.89   2710.25
Lost                    34.92       1.00    325.62
Loyal Customers         22.61       1.54    578.19
New/Recent Customers    13.66       1.00    466.64


In [12]:
check = df_clean.groupby('Segment').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean',
    'CustomerID': 'count'
}).round(2)

print(check.sort_values(by='monetary', ascending=False))

                      recency  frequency  monetary  CustomerID
Segment                                                       
Champions               11.06       3.89   2710.25         215
At Risk                 34.27       2.03    904.38          98
Loyal Customers         22.61       1.54    578.19         180
New/Recent Customers    13.66       1.00    466.64         130
Average/Potential       31.25       1.00    355.24         168
Lost                    34.92       1.00    325.62         183


In [13]:
df_clean['monetary'] = df_clean['monetary'].round(2)

In [14]:
df_final = df.dropna(subset=['CustomerID'])

In [19]:
df_final.to_csv('rfm_segmented_final.csv', index=False)
print("Success!")

Success!


In [20]:
df_final = df[df['CustomerID'].notnull()]

In [21]:
df_final.to_csv('rfm_segmented_final.csv', index=False)
print("Success!")

Success!


In [23]:
import numpy as np
df['CustomerID'] = df['CustomerID'].replace('', np.nan)

In [24]:
df_final = df.dropna(subset=['CustomerID'])

In [25]:
print("New Top Spenders (Should NOT include a blank ID):")
print(df_final.sort_values(by='monetary', ascending=False).head())

New Top Spenders (Should NOT include a blank ID):
    CustomerID  recency  frequency  monetary R_Score F_Score M_Score  \
950      18102       31          4  27834.61       2       5       5   
416      15061       25         12  19950.66       3       5       5   
579      16029        5          7  13202.52       5       5       5   
826      17511        4          4  10573.22       5       5       5   
345      14646       19          2   8591.88       4       4       5   

             Segment  
950          At Risk  
416  Loyal Customers  
579        Champions  
826        Champions  
345        Champions  


In [26]:
df_final.to_csv('rfm_segmented_final.csv', index=False)